In [ ]:
import os
import driver
import numpy as np
from pynq import Overlay
from PIL import Image,ImageFile

In [ ]:
conv2d_0_kernel  = np.load('../data_npy/pynq_net_conv2d_kernel_0.npy')
conv2d_0_bias    = np.load('../data_npy/pynq_net_conv2d_bias_0.npy')

conv2d_1_kernel  = np.load('../data_npy/pynq_net_conv2d_1_kernel_0.npy')
conv2d_1_bias    = np.load('../data_npy/pynq_net_conv2d_1_bias_0.npy')

conv2d_2_kernel  = np.load('../data_npy/pynq_net_conv2d_2_kernel_0.npy')
conv2d_2_bias    = np.load('../data_npy/pynq_net_conv2d_2_bias_0.npy')

conv2d_3_kernel  = np.load('../data_npy/pynq_net_conv2d_3_kernel_0.npy')
conv2d_3_bias    = np.load('../data_npy/pynq_net_conv2d_3_bias_0.npy')

conv2d_4_kernel  = np.load('../data_npy/pynq_net_conv2d_4_kernel_0.npy')
conv2d_4_bias    = np.load('../data_npy/pynq_net_conv2d_4_bias_0.npy')

dense_0_kernel   = np.load('../data_npy/pynq_net_dense_kernel_0.npy')
dense_0_bias     = np.load('../data_npy/pynq_net_dense_bias_0.npy')

dense_1_kernel   = np.load('../data_npy/pynq_net_dense_1_kernel_0.npy')
dense_1_bias     = np.load('../data_npy/pynq_net_dense_1_bias_0.npy')

dense_2_kernel   = np.load('../data_npy/fusionnet_dense_kernel_0.npy')
dense_2_bias     = np.load('../data_npy/fusionnet_dense_bias_0.npy')

In [ ]:
overlay = Overlay("fusion.bit")

In [ ]:
quant_scale=4096
overlay.cnn.load_weight_conv(conv2d_0_kernel,conv2d_0_bias, 1, quant_scale,10)
overlay.cnn.load_weight_conv(conv2d_1_kernel,conv2d_1_bias, 2, quant_scale, 1)
overlay.cnn.load_weight_conv(conv2d_2_kernel,conv2d_2_bias, 3, quant_scale, 1)
overlay.cnn.load_weight_conv(conv2d_3_kernel,conv2d_3_bias, 4, quant_scale, 1)
overlay.cnn.load_weight_conv(conv2d_4_kernel,conv2d_4_bias, 5, quant_scale, 1)
overlay.cnn.load_weight_fc(dense_0_kernel,dense_0_bias, 6, quant_scale, 1)
overlay.cnn.load_weight_fc(dense_1_kernel,dense_1_bias, 7, quant_scale, 1)

overlay.mlp.load_weight_fc_mlp(dense_2_kernel,dense_2_bias, 6, quant_scale, 1)

In [ ]:
def generateds(img_path):
    img = Image.open(img_path)  # 读入图片
    img = np.array(img.convert('L'))  # 图片变为8位宽灰度值的np.array格式
    img   = img / 10.0
    x = []
    x.append(img)  # 归一化后的数据，贴到列表x
    x = np.array(x)  # 变为np.array格式
    x = x[:,:,:,np.newaxis]
    return x 

In [ ]:
x = generateds('./5_1.jpg')

In [ ]:
result_vision = overlay.cnn.execute_s(x,1,256,10,1)

In [ ]:
result_vision = result_vision.reshape((1,10))

In [ ]:
leap_input = np.load('../data_npy/leap.npy')

In [ ]:
leap_input = leap_input.reshape(1,2)

In [ ]:
input_fusion = np.hstack((result_vision,leap_input))

In [ ]:
overlay.mlp.execute_s_mlp(input_fusion,12,1,10,1)